In [ ]:
import numpy as np
from ian.ian import *
from ian.utils import *
from ian.dset_utils import *
from ian.embed_utils import *
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams['figure.figsize']  = (6.0, 4.0)
plt.rcParams['figure.dpi'] = 72.0

## Create dataset

In [ ]:
### Sample points uniformly at random from a 5-d cylinder (S^4 x R^1) or radius 1 and length 3 in R^6  

np.random.seed(1)


d = 5 #ambient space for the 4-sphere
length = 3 #cylinder length along its main axis
n = 7 #sets the sampling density

# sample using Muller method
# (e.g., see http://extremelearning.com.au/how-to-generate-uniformly-random-points-on-n-spheres-and-n-balls/)
u = np.random.normal(0, 1, (n**d,d))  # an array of d normally distributed random variables
norm = np.linalg.norm(u,axis=1,keepdims=1)
X = u/norm #all pts become unit-norm (unit radius sphere)

#sample positions along cylinder length in R^1, adding an extra dimension
X = np.concatenate([np.random.random(n**d)[:,None]*length,X],axis=1)
X -= X.mean(0)
N = X.shape[0]

#subsample: using half the total points, for speed
frac = .5 #fraction to use
subs = np.random.choice(range(N),int(N*frac),False)
X = X[subs]
N = X.shape[0]
print(X.shape)

In [ ]:
### setup color labels

red = tuple([1,0,0,1])
yellow = tuple([1,1,0,1])
black = tuple([0,0,0,1])
green = tuple([73/255,252/255,3/255])
magenta = tuple([1,0,1,1])
blue = tuple([0,0,1,1])

clrs_to_use = [red,yellow,black,green,magenta,blue]
nclrs = len(clrs_to_use)

clr_thickness = X[:,0].ptp()/nclrs
h0 = X[:,0].min()
clbls = np.zeros(N)
cclrs = np.zeros((N,3))
for clri in range(nclrs):
    mask = (X[:,0] >= h0 + clri*clr_thickness) & (X[:,0] <= h0 + (clri+1)*clr_thickness)
    clbls[mask] = clri
    cclrs[mask] = np.tile(clrs_to_use[clri][:3],(mask.sum(),1))

f,ax = subps(1,1,3,5)
plot2dScatter(X[:,[0,1]],cclrs,s=50,axLabels=False,alpha=1,edgecolors='gray',f_ax=(f,ax),axisEqual=True)
ax.set_title('original data')
plt.show()

In [ ]:
#plot labels of each point sorted by position along first principal component (main axis) in embedding
def showPositionalLabelMixing(y,titl=''):
    x_height_order = np.argsort(X[:,0])
    if y[x_height_order[0]] > y[x_height_order[-1]]:#check for flipped sign compared to original data
        y = -y
    height_order = np.argsort(y)
    n_pts_per_color = N//nclrs

    f, ax = subps(1,1,4,6)
    ax.plot(clbls[height_order])
    for clri in range(nclrs):
        start,end = clri*n_pts_per_color, (clri+1)*n_pts_per_color
        rect = plt.Rectangle((start,0), end-start, nclrs-1,
                                     facecolor=clrs_to_use[clri], alpha=0.3)
        ax.add_patch(rect)
    ax.set(xlabel='index along main axis in the embedding', ylabel='original label (cylinder section)', title=titl)
    plt.show()


# compute Kendall's tau to assess degree of mixing caused by the embedding algorithm
# for original data it is trivially 1.00 (100% match)

def plot_tau_stats(y, titl=''):
    x_height_order = np.argsort(X[:,0])
    if y[x_height_order[0]] > y[x_height_order[-1]]:
        y = -y
    xmap = {v:k for k,v in enumerate(x_height_order)}
    y_height_order = [xmap[yi] for yi in np.argsort(y)]

    tau = sp.stats.kendalltau(np.arange(N),y_height_order)[0]
    
    f, ax = subps(1,1,3,3)
    ax.plot(y_height_order)
    ax.text(.1,.9,'tau = %.2f' % tau,size=14,transform=ax.transAxes)
    ax.set_xlabel(r'horiz. ranking in $\mathcal{X}$')
    ax.set_ylabel(r'horiz. ranking in $\mathcal{X}$')
    ax.set_title(titl)
    ax.set(xticks=[], yticks=[])
    plt.show()
    
    return tau

y = X[:,0]
algo_title = 'Original data'
showPositionalLabelMixing(y, algo_title)
tau = plot_tau_stats(y, algo_title)

## Compute IAN kernel

In [ ]:
D2 = pwdists(X, sqdists=True) #square matrix of squared Euclidean distances
D1 = np.sqrt(D2)

In [ ]:
#using greedy approach for speed (took me about 43 seconds)
G, wG, optScales, disc_pts = IAN('exact-precomputed-sq', D2, obj='greedy')

# for higher accuracy, use linear program (took me about 12 minutes using the GUROBI solver),
# but resulting embeddings are very similar
# G, wG, optScales, disc_pts = IAN('exact-precomputed-sq', D2, obj='l1', solver='GUROBI')

In [ ]:
#use average node degree as a reasonable number k of nearest neighbors to use with t-SNE (perplexity) and UMAP (n_neighbors)
avg_deg = int(G.toarray().sum(1).mean()) #parameter must be an integer
print('Average degree in unweighted graph:', avg_deg)

## Compare embeddings using different algorithms

In [ ]:
n_components = 2
rs = 0 #same random state to use across algorithms

### Diffusion maps

In [ ]:
algo_title = 'IAN + diffusion maps'

diffmap_y, lams = diffusionMapFromK(wG, n_components)

#use PCA to align cylinder's main axis with first component, and allow for a fair comparison
dmPCs = PCA(2, random_state=rs).fit_transform(diffmap_y)
dmPCs[:,0] = -dmPCs[:,0] #(flipping in order to have red points on the left)
f,ax = subps(1,1,3,5)
plot2dScatter(dmPCs[:,[0,1]],cclrs,s=50,axLabels=False,edgecolors='gray',f_ax=(f,ax),axisEqual=False)
ax.set_title(algo_title)
plt.show()

In [ ]:
y = dmPCs[:,0]
showPositionalLabelMixing(y, algo_title)
tau = plot_tau_stats(y, algo_title)

### t-SNE

(note: the t-SNE implementation used here is included in the scikit-learn package)

In [ ]:
algo_title = 't-SNE, k = %d' % avg_deg

knn = avg_deg

# 1 - use same initialization method used by umap (spectral embedding, i.e. Laplacian eigenmaps)
from sklearn.manifold import SpectralEmbedding
spec = SpectralEmbedding(n_components=n_components,affinity='precomputed',random_state=rs)
# (need to compute a kernel matrix first, using t-SNE's own sigmas) 
sigmas = computeTSNEsigmas(D2, knn) #use knn as perplexity
myK = computeTSNEkernel(D2, sigmas, False) #no need to normalize b/c will be done by eigenmaps
init = spec.fit_transform(myK)

# 2 - compute t-SNE embedding with the init above
tsne = TSNE(n_components=n_components, learning_rate='auto',metric='precomputed',
           init=init, perplexity=knn, random_state=rs, method="barnes_hut", verbose=1)
ty = tsne.fit_transform(D1)


In [ ]:
#use PCA to align main axis and allow for a fair comparison
tPCs = PCA(2, random_state=rs).fit_transform(ty)[:,:2]
f,ax = subps(1,1,3,5)
plot2dScatter(tPCs,cclrs,s=40,axLabels=False,edgecolors='gray',f_ax=(f,ax),axisEqual=False)
ax.set_title(algo_title)

In [ ]:
y = tPCs[:,0]
showPositionalLabelMixing(y, algo_title)
tau = plot_tau_stats(y, algo_title)

In [ ]:
#### IAN + t-SNE
# use IAN's optimal scales in the t-SNE kernel
algo_title = 'IAN + t-SNE'

# 1 - use same initialization method used by umap (spectral embedding, i.e. Laplacian eigenmaps)
from sklearn.manifold import SpectralEmbedding
spec = SpectralEmbedding(n_components=n_components,affinity='precomputed',random_state=rs)
myK = computeTSNEkernel(D2, optScales, False) #no need to normalize b/c will be done by eigenmaps
init = spec.fit_transform(myK)

# 2 - compute t-SNE embedding with the init above
tsne = TSNE(n_components=n_components, learning_rate='auto', metric='precomputed', #setting perplexity here is irrelevant, won't be used
               init=init, random_state=rs, method="barnes_hut", verbose=1)
ity = my_tsne_fit_transform(tsne,D2.copy(),optScales)

In [ ]:
#use PCA to align main axis and allow for a fair comparison
itPCs = PCA(2, random_state=rs).fit_transform(ity)[:,:2]
f,ax = subps(1,1,3,5)
plot2dScatter(itPCs,cclrs,s=40,axLabels=False,edgecolors='gray',f_ax=(f,ax),axisEqual=False)
ax.set_title(algo_title)

In [ ]:
y = itPCs[:,0]
showPositionalLabelMixing(y, algo_title)
tau = plot_tau_stats(y, algo_title)

### UMAP

(note: UMAP is not a pre-requisite for the IAN package; you will need to install it independently in order to run the code below)

In [ ]:
import umap #tested using umap-learn v0.5.3

algo_title = 'UMAP, k = %d' % avg_deg

fit = umap.UMAP(
    n_neighbors=avg_deg,
    n_components=n_components,
    random_state=rs,
)
uY = fit.fit_transform(X)

#use PCA to align cylinder's main axis with first component, and allow for a fair comparison
uPCs = PCA(2, random_state=rs).fit_transform(uY)[:,:2]
f,ax = subps(1,1,3,5)
plot2dScatter(uPCs,cclrs,s=40,axLabels=False,edgecolors='gray',f_ax=(f,ax),axisEqual=False)
ax.set_title(algo_title)
plt.show()

In [ ]:
y = uPCs[:,0]
showPositionalLabelMixing(y, algo_title)
tau = plot_tau_stats(y, algo_title)

In [ ]:
### IAN + UMAP

algo_title = 'IAN + UMAP'

#setting up a umap object
u = umap.UMAP(
    n_neighbors=avg_deg,
    n_components=n_components,
    transform_mode="graph", #this option tells umap not to compute the embedding just yet
    random_state=rs,
)
u.fit(X)

#run algorithm with our own weighted graph
myK = wG.toarray()
np.fill_diagonal(myK,0) #remove self-weights (not used by umap)
sparseK = sp.sparse.coo_matrix(myK) #convert into sparse COO format, required by umap


#now run umap's embedding algorithm directly, using the IAN weighted graph
from sklearn.utils import check_random_state, check_array
X_ = check_array(X.copy(), dtype=np.float32, accept_sparse="csr", order="C")

from umap.umap_ import simplicial_set_embedding
uY,_ = simplicial_set_embedding(X_, sparseK, #just pass along all internal attributes from the umap object create above
            u.n_components, u._initial_alpha, u._a, u._b, u.repulsion_strength, u.negative_sample_rate,
            u.n_epochs, u.init, check_random_state(u.random_state), u._input_distance_func, u._metric_kwds,
            u.densmap, u._densmap_kwds, u.output_dens, u._output_distance_func, u._output_metric_kwds,
            u.output_metric in ("euclidean", "l2"), check_random_state(u.random_state) is None, u.verbose, u.tqdm_kwds)

#use PCA to align main axis, allowing for a fair comparison
iuPCs = PCA(2,random_state=rs).fit_transform(uY)
iuPCs[:,0] = -iuPCs[:,0] #(flipping in order to have red points on the left)
f,ax = subps(1,1,3,5)
plot2dScatter(iuPCs[:,[0,1]],cclrs,s=40,axLabels=False,edgecolors='gray',f_ax=(f,ax),axisEqual=False)
ax.set_title(algo_title)
plt.show()

In [ ]:
y = iuPCs[:,0]
showPositionalLabelMixing(y, algo_title)
tau = plot_tau_stats(y, algo_title)